# Assignment 1.2: Word2vec preprocessing (20 points)

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make **Batcher** class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size, 2*window_size)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']
```

In [31]:
import nltk
import numpy as np
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
# for SkipGram
class Batcher:
  def preprocess(self):
      self.tokens = nltk.word_tokenize(self.raw_text)
      # self.tokens = self.raw_text
      self.corpus_size = len(self.tokens)

      counter = Counter(self.tokens)
      counter = {k: v for k, v in sorted(counter.items(),
                                              key=lambda item: item[1])}

      most_frequent = list(counter)[:min(len(counter), self.vocab_size - 1)]
      
      self.word2Ind = {word : ind for ind, word in enumerate(most_frequent)}
      self.word2Ind[self.unknown] = len(self.word2Ind)
      self.ind2Word = {v : k for k, v in self.word2Ind.items()}

      self.tokens = np.array([self.word2Ind[token] if token in most_frequent 
                     else self.word2Ind[self.unknown]
                     for token in self.tokens])

      # обрезаем текст чтобы все батчи имели один шейп
      self.shuffle = np.arange(self.window_size, len(self.tokens)
       - self.window_size)
      np.random.shuffle(self.shuffle)
      

  def __init__(self, window_size=2, batch_size=100,
               data_path = 'text8', vocab_size = 10000):
    self.window_size = window_size
    self.batch_size = batch_size
    self.vocab_size = vocab_size
    self.unknown = '<UNK>'

    self.raw_text = open(data_path).read()
    # self.raw_text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']
    self.preprocess()
    
    self.ind = 0 

    
  def __next__(self):
    # если последний батч вылезает за границы корпуса, просто пропускаем его
    if self.ind + self.batch_size >= len(self.shuffle):
      raise StopIteration

    centers_pos = self.shuffle[self.ind:self.ind + self.batch_size]
    
    windows = [list(self.tokens[c - self.window_size:c]) + list(
                                 self.tokens[c + 1: c + self.window_size + 1])
                                 for c in centers_pos]

    self.ind += self.batch_size
    return self.tokens[centers_pos], np.array(windows)
    

In [0]:
batcher = Batcher(batch_size = 5, window_size=2, vocab_size=100000)

In [30]:
print(next(batcher))

(array([16972, 16181, 12236, 16785, 17046]), array([[17249, 15947, 17249, 15546],
       [13737, 13204, 16388, 17152],
       [16483, 17017, 17083, 17246],
       [17225, 17249, 17248, 14955],
       [17242, 17234, 11277, 17244]]))
